# Load Data & Train Model
This notebook allows us to load the entire dataset, split it into a proper train/test split and trains the model using K-Fold cross validation.

## Imports

In [1]:
# Essentials
import math
import random
import os
import copy
import glob
import shutil
import numpy as np
import json

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential as Seq, Linear as Lin, Conv2d

import torchvision.models as models
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import timm
from timm.models import create_model
from timm.data import create_transform
from sklearn.metrics import accuracy_score

# Images
import albumentations
import albumentations.pytorch

import cv2

from PIL import Image

# Machine Learning
from sklearn.model_selection import KFold

from barbar import Bar

## Set Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
torch.cuda.device_count()

2

In [4]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

## Load Dataset

This downloads the dataset to the server and unzips it so we can use it. Check the size of the folder before running the rest of the code. Should be around 4GB

In [18]:
!wget http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz

--2024-01-05 15:52:56--  http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Resolving www.inf.ufpr.br (www.inf.ufpr.br)... 200.17.202.113, 2801:82:80ff:8001:216:ccff:feaa:79
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz [following]
--2024-01-05 15:52:57--  https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273561758 (4.0G) [application/octet-stream]
Saving to: ‘BreaKHis_v1.tar.gz’

BreaKHis_v1.tar.gz  100%[===================>]   3.98G  17.7MB/s    in 3m 55s  

2024-01-05 15:56:54 (17.3 MB/s) - ‘BreaKHis_v1.tar.gz’ saved [4273561758/4273561758]



In [19]:
!tar -xf BreaKHis_v1.tar.gz

In [5]:
!du -sh ./BreaKHis_v1

4.1G	./BreaKHis_v1


In [20]:
!du -sh ./Master.pth

du: cannot access './Master.pth': No such file or directory


In [5]:
full_dataset = './BreaKHis_v1/histology_slides/breast/**/SOB/**/**/**/*.png'

## Prepare Data
In this section we do the following:
1. Store all image paths in a dictionary that is filtered on class and zoom level
2. Create a stratified train/test split (based on class and zoom level) and store the image paths of both sets again in a filtered dictionary
3. Copy all images from the raw data source to the structured data folder "dataset/"
4. Create a 5-fold cross validation split for the train dataset
5. Prepare all splits for forward pass of the model

### Dictionary

In [6]:
# Store all 7909 filepaths in a list
all_files = glob.glob(full_dataset)

# All eight classes
classes = ["A", "F", "TA", "PT", "DC", "LC", "MC", "PC"]
benign_classes = classes[:4]
malignant_classes = classes[4:]

# All four zoom levels
zooms = ["40", "100", "200", "400"]

# Create dictionary that filters images based on class and zoom level
data_dict = {c: {z: [path for path in all_files if path.split("_")[-1].split("-")[0] == c and path.split("_")[-1].split("-")[3] == z] for z in zooms} for c in classes}

# For demonstration purposes a splitted file path
# We need first element (class) and fourth element (zoom level)
print("Splitted file path:", all_files[0].split("_")[-1].split("-"))

# Check number of items per class per zoom level
for c in data_dict.keys():
    for z, v in data_dict[c].items():
        print(c, z, len(v))

Splitted file path: ['DC', '14', '20629', '100', '027.png']
A 40 114
A 100 113
A 200 111
A 400 106
F 40 253
F 100 260
F 200 264
F 400 237
TA 40 149
TA 100 150
TA 200 140
TA 400 130
PT 40 109
PT 100 121
PT 200 108
PT 400 115
DC 40 864
DC 100 903
DC 200 896
DC 400 788
LC 40 156
LC 100 170
LC 200 163
LC 400 137
MC 40 205
MC 100 222
MC 200 196
MC 400 169
PC 40 145
PC 100 142
PC 200 135
PC 400 138


### Train/Test Split

In [7]:
# Initialize empty dicts
train_dict = {c: {z: [] for z in zooms} for c in classes}
test_dict = {c: {z: [] for z in zooms} for c in classes}

# Make train/test split
train_test_split = 0.9

for c in data_dict.keys():
    for z, v in data_dict[c].items():
        split = math.ceil(train_test_split * len(v))

        # Randomly sample from file paths for given class/zoom level
        train_images = random.sample(data_dict[c][z], split)

        # Take as test data all files that are not in the train data for a given class/zoom level
        test_images = np.setdiff1d(data_dict[c][z], train_images)

        # Check if error is made
        if len(train_images) + len(test_images) != len(v):
            print("Error in train/test split at {}-{}".format(c, z))

        # Store train and test data in dictionaries
        train_dict[c][z] = list(train_images)
        test_dict[c][z] = list(test_images)

In [8]:
# Check if splitting went correct

# Print number of items in train and test data for class and zoom level
for c in classes:
    for z in zooms:
        print("Class {}, Zoom {}".format(c, z))
        print("Total: {}, Train: {}, Test: {}".format(len(data_dict[c][z]), len(train_dict[c][z]), len(test_dict[c][z])))

Class A, Zoom 40
Total: 114, Train: 103, Test: 11
Class A, Zoom 100
Total: 113, Train: 102, Test: 11
Class A, Zoom 200
Total: 111, Train: 100, Test: 11
Class A, Zoom 400
Total: 106, Train: 96, Test: 10
Class F, Zoom 40
Total: 253, Train: 228, Test: 25
Class F, Zoom 100
Total: 260, Train: 234, Test: 26
Class F, Zoom 200
Total: 264, Train: 238, Test: 26
Class F, Zoom 400
Total: 237, Train: 214, Test: 23
Class TA, Zoom 40
Total: 149, Train: 135, Test: 14
Class TA, Zoom 100
Total: 150, Train: 135, Test: 15
Class TA, Zoom 200
Total: 140, Train: 126, Test: 14
Class TA, Zoom 400
Total: 130, Train: 117, Test: 13
Class PT, Zoom 40
Total: 109, Train: 99, Test: 10
Class PT, Zoom 100
Total: 121, Train: 109, Test: 12
Class PT, Zoom 200
Total: 108, Train: 98, Test: 10
Class PT, Zoom 400
Total: 115, Train: 104, Test: 11
Class DC, Zoom 40
Total: 864, Train: 778, Test: 86
Class DC, Zoom 100
Total: 903, Train: 813, Test: 90
Class DC, Zoom 200
Total: 896, Train: 807, Test: 89
Class DC, Zoom 400
Total: 78

Finally, we dump the train and test dictionaries in a json file. This allows us to pass files more easily between projects.

In [9]:
# Create train text file
with open("txt/train.txt", "w") as f:
    json.dump(train_dict, f)

# Create test text file
with open("txt/test.txt", "w") as f:
    f.write(json.dumps(test_dict))

### Copy Images

In [17]:
# Takes all images from a train/test dict and copies them to new destination for easy access
def copy_images(image_dict, type):

    # Get current working directory
    current_dir = os.getcwd()

    # Check whether the new file structure is already initialised
    if not os.path.isdir(os.path.join(current_dir, "dataset", "train", "TA", "400")) or not os.path.isdir(os.path.join(current_dir, "dataset", "test", "TA", "400")):
        for c in classes:
            for z in zooms:
                os.makedirs(os.path.join(os.getcwd(), "dataset", "test", c, z))
                os.makedirs(os.path.join(os.getcwd(), "dataset", "train", c, z))

    for c in classes:
        for z in zooms:

            # Destination path
            dst = os.path.join(current_dir, "dataset", type, c, z)

            # Remove all images currently in destination path
            for item in os.listdir(dst):
                item_path = os.path.join(dst, item)
                if os.path.isfile(item_path):
                    os.remove(item_path)
                elif os.path.isdir(item_path):
                    # Use rmtree for directories
                    shutil.rmtree(item_path)

            # Copy all images from BreaKHis to dataset directory
            for image in image_dict[c][z]:
                src = os.path.join(image)
                dst = os.path.join(current_dir, "dataset", type, c, z)

                shutil.copy2(src, dst)

In [18]:
# Copy images to correct directory for easy access
copy_images(test_dict, "test")
copy_images(train_dict, "train")

In [19]:
# Check if copying went correct

# Print all images of class LC and zoom 100 in train dict
# Compare this with images in dataset/train/LC/100
for name in sorted(train_dict["LC"]["100"]):
    print(name)

./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-031.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-032.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-034.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-035.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-036.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-037.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-038.png
./BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-12204/100X/SOB_M_LC-14-12204-100-039.png
./BreaKHis_v1/hi

### K-Fold Split
Explanation of K-Fold: https://isheunesu48.medium.com/cross-validation-using-k-fold-with-scikit-learn-cfc44bf1ce6

In [20]:
# Determine number of folds
n_folds = 5

# Store train and validate for every fold
folds = {str(i): {"train": [], "validate": []} for i in range(n_folds)}

kf = KFold(n_splits=n_folds)

for c in classes:
    for z in zooms:

        # For every class and zoom, create a 5-fold split
        for i, (train_index, validate_index) in enumerate(kf.split(train_dict[c][z])):

            # Store paths of all train images in fold i
            fold_train_img = [img for i, img in enumerate(train_dict[c][z]) if i in train_index]

            # Store paths of all validate images in fold i
            fold_validate_img = [img for i, img in enumerate(train_dict[c][z]) if i in validate_index]

            # Add paths to fold i
            folds[str(i)]["train"] += fold_train_img
            folds[str(i)]["validate"] += fold_validate_img

# Shuffle images in each train/validate fold to make sure order of classes is mixed
for i in range(n_folds):
    random.shuffle(folds[str(i)]["train"])
    random.shuffle(folds[str(i)]["validate"])

# Check number of train and validate items per fold
for k, v in folds.items():
    print("Fold", int(k)+1)
    print("Train:", len(v["train"]))
    print("Validate:", len(v["validate"]))
    print()

Fold 1
Train: 5694
Validate: 1438

Fold 2
Train: 5698
Validate: 1434

Fold 3
Train: 5704
Validate: 1428

Fold 4
Train: 5713
Validate: 1419

Fold 5
Train: 5719
Validate: 1413



In [21]:
# Check for duplicates in validation set
sets = []
for i in range(n_folds):
    sets.append(set(folds[str(i)]["validate"]))

for i in sets:
    for j in sets:
        if i == j:
            continue

        duplicates = i.intersection(j)
        if duplicates:
            print("Duplicates found:", len(duplicates))

### Prepare Data for Forward Pass

In [22]:
class My_data(Dataset):
    def __init__(self, data, transforms=None):
        self.image_list = data
        self.data_len = len(self.image_list)
        self.transforms = transforms
        self.eicls = ["A", "F", "TA", "PT", "DC", "LC", "MC", "PC"]

    def __getitem__(self, index):       
        current_image_path = self.image_list[index]
        im_as_im = cv2.imread(current_image_path)
        if im_as_im is None:
            raise ValueError(f"Image not found or is corrupted: {current_image_path}")
        im_as_im = cv2.cvtColor(im_as_im, cv2.COLOR_BGR2RGB)

        # Perform label encoding for multi-label classification
        parts = current_image_path.split('_')[-1].split('-')
        if parts[2]=="13412":
            labels =[0,0,0,0,1,1,0,0]
        else:
            labels = [int(label == parts[0]) for label in self.eicls]
        labels = torch.tensor(labels)

        if self.transforms is not None:
            augmented = self.transforms(image=im_as_im)
            im_as_im = augmented['image']

        return (im_as_im, labels)

    def __len__(self):
        return self.data_len

In [23]:
transform = {
    'train': albumentations.Compose([
    albumentations.Resize(256, 256),
    albumentations.OneOf([
                          albumentations.HorizontalFlip(),
                          albumentations.Rotate(limit=45),
                          albumentations.VerticalFlip(),
                          albumentations.GaussianBlur(),
                          albumentations.NoOp()
    ], p=1),
    albumentations.Normalize(mean=[0, 0, 0], std=[255, 255, 255], max_pixel_value=1.0),
    albumentations.pytorch.transforms.ToTensorV2()]),

    'valid': albumentations.Compose([
    albumentations.Resize(256, 256),
    albumentations.Normalize(mean=[0, 0, 0], std=[255, 255, 255], max_pixel_value=1.0), 
    albumentations.pytorch.transforms.ToTensorV2()]),
}

In [24]:
# Create datasets for each fold
train_folds = [My_data(folds[str(i)]["train"], transforms=transform['train']) for i in range(n_folds)]
validate_folds = [My_data(folds[str(i)]["validate"], transforms=transform['valid']) for i in range(n_folds)]

# Create data loaders for each fold
train_dataloaders = [DataLoader(dataset=train_folds[i], batch_size=16,shuffle=True,num_workers=2,
                                              pin_memory=True,prefetch_factor=2) for i in range(n_folds)]

validate_dataloaders = [DataLoader(dataset=validate_folds[i], batch_size=16,shuffle=True,num_workers=2,
                                              pin_memory=True,prefetch_factor=2) for i in range(n_folds)]

## Create Model

In [20]:
# Number of samples in each class
# Assumption is made that in each fold, these are the same (since we do stratified split)
class_samples = [sum([len(v) for v in z.values()]) for z in train_dict.values()]

# Hardcoded from paper
# class_samples = [367, 803, 456, 370, 2763, 492, 629, 449]  # Number of samples in each class for training

total_samples = sum(class_samples)
samples = total_samples/len(class_samples)
class_weights = [samples / (s + 1e-8) for s in class_samples]
class_weights = torch.tensor(class_weights)
print(class_weights)

print(sum(class_samples))

tensor([2.2232, 0.9754, 1.7378, 2.1744, 0.2868, 1.5779, 1.2469, 1.7619])
7132


In [27]:
model = timm.create_model(
    'swinv2_tiny_window8_256.ms_in1k',
    pretrained=True,
    features_only=False,
    num_classes=8,
    # TODO: find out what these mean
    drop_path_rate=0.2,
    drop_rate=0.5
)

print(model.get_classifier())
model

/home/jovyan/.local/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Linear(in_features=768, out_features=8, bias=True)


SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1): Identi

In [28]:
print(len([param for param in model.named_parameters()]))

# Iterate over the parameters and check requires_grad
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter '{name}' requires grad.")
    else:
        print(f"Parameter '{name}' does not require grad.")
    
    param.requires_grad = True

221
Parameter 'patch_embed.proj.weight' requires grad.
Parameter 'patch_embed.proj.bias' requires grad.
Parameter 'patch_embed.norm.weight' requires grad.
Parameter 'patch_embed.norm.bias' requires grad.
Parameter 'layers.0.blocks.0.attn.logit_scale' requires grad.
Parameter 'layers.0.blocks.0.attn.q_bias' requires grad.
Parameter 'layers.0.blocks.0.attn.v_bias' requires grad.
Parameter 'layers.0.blocks.0.attn.cpb_mlp.0.weight' requires grad.
Parameter 'layers.0.blocks.0.attn.cpb_mlp.0.bias' requires grad.
Parameter 'layers.0.blocks.0.attn.cpb_mlp.2.weight' requires grad.
Parameter 'layers.0.blocks.0.attn.qkv.weight' requires grad.
Parameter 'layers.0.blocks.0.attn.proj.weight' requires grad.
Parameter 'layers.0.blocks.0.attn.proj.bias' requires grad.
Parameter 'layers.0.blocks.0.norm1.weight' requires grad.
Parameter 'layers.0.blocks.0.norm1.bias' requires grad.
Parameter 'layers.0.blocks.0.mlp.fc1.weight' requires grad.
Parameter 'layers.0.blocks.0.mlp.fc1.bias' requires grad.
Parame

In [30]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, class_weights=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.class_weights = class_weights

    def forward(self, logits, labels):
        probs = torch.sigmoid(logits)
        ce_loss = nn.BCELoss()(probs, labels)
        weight = (1 - probs).pow(self.gamma)
        loss = ce_loss  # Initialize loss with cross-entropy loss
        if self.class_weights is not None:
            weight = weight * self.class_weights
            loss = loss * weight
        return loss

In [31]:
model = model.to(device)
class_weights = class_weights.to(device)

criterion = FocalLoss(class_weights)

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

best_model_wts = model.state_dict()
best_optimizer_state =optimizer.state_dict()
best_acc = 0.0

## Train Model

This section performs the actual training of the model. We first determine methods fit and validate that will be called during the training. Afterwards, we define the loop that optimizes the model.

In [32]:
def fit(model, dataloader, optimizer,scheduler, criterion):
    #print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    accum_iter = 4

    for i, (inputs, labels) in enumerate(Bar(dataloader)):
        inputs = inputs.to(device)
        labels = labels.float().to(device)
        optimizer.zero_grad()
        #model.zero_grad(set_to_none=True)
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)
        thresholds = [0.5, 0.5, 0.5,0.5,0.5,0.5,0.5,0.5]
        # Compute loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()* inputs.size(0)
        # _ , preds = torch.max(outputs.data, 1)
        # Apply sigmoid activation to obtain probabilities
        #preds = (outputs > 0.5).float()
        probs = torch.sigmoid(outputs)
        preds = torch.zeros_like(probs)

        # Set predicted labels based on the threshold
        for i, threshold in enumerate(thresholds):
            preds[:, i] = (probs[:, i] >= threshold).float()
        train_running_correct += (preds == labels).all(dim=1).float().sum()
        # Backpropagate the gradients
        loss /= accum_iter
        loss.backward()

        if ((i + 1) % accum_iter == 0) :
            optimizer.step()
            optimizer.zero_grad()

    scheduler.step()

    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(dataloader.dataset)
    return train_loss, train_accuracy

In [33]:
def validate(model, dataloader, optimizer, criterion):
    #print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.float()
            labels = labels.to(device)
            outputs = model(inputs)
            thresholds = [0.5, 0.5, 0.5,0.5,0.5,0.5,0.5,0.5]
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()*inputs.size(0)
            #_, preds = torch.max(outputs.data, 1)
            #preds = (outputs > 0.5).float()
            probs = torch.sigmoid(outputs)
            preds = torch.zeros_like(probs)
            # Set predicted labels based on the threshold
            for i, threshold in enumerate(thresholds):
                preds[:, i] = (probs[:, i] >= threshold).float()
            val_running_correct += (preds == labels).all(dim=1).float().sum()

    val_loss = val_running_loss/len(dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
    return val_loss, val_accuracy

In [35]:
import time as time
history=[]
best_model_wts = copy.deepcopy(model.state_dict())
#best_optimizer_state = copy.deepcopy(optimizer.state_dict())
best_acc = 0.0
epochs=50

train_dataloader = train_dataloaders[0]
validate_dataloader = validate_dataloaders[0]

for epoch in range(epochs):
    epoch_start = time.time()
    print('Epoch-{0}/{1} lr: {2}'.format(epoch+1,epochs ,optimizer.param_groups[0]['lr']))
    
    if epoch % 10 == 0:
        train_dataloader = train_dataloaders[int(epoch / 10)]
        valid_dataloader = validate_dataloaders[int(epoch / 10)]

    # Why is this here???
    if  epoch > 14:
        for param in model.parameters():
            param.requires_grad = True
    #print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_accuracy = fit(model,train_dataloader,optimizer,scheduler,criterion)
    val_epoch_loss, val_epoch_accuracy = validate(model,valid_dataloader,optimizer,criterion)

    epoch_end = time.time()
    history.append([epoch+1,train_epoch_loss, train_epoch_accuracy, val_epoch_loss, val_epoch_accuracy,(epoch_end-epoch_start)])
    print(f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f},Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_accuracy:.2f},time : {epoch_end-epoch_start:.2f}")
    torch.save({'history':history},'Master_his.pth')
    if val_epoch_accuracy > best_acc:
        best_acc = val_epoch_accuracy
        best_model_wts = copy.deepcopy(model.state_dict())

        best_epoch=epoch
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': best_model_wts,
            'loss': criterion,
            'history':history,
            'best_epoch': best_epoch+1,

            }, 'Master.pth')

Epoch-1/50 lr: 0.0001


5694/5694: [===============================>] - ETA 0.7sss
Train Loss: 0.1558, Train Acc: 63.52,Val Loss: 0.0885, Val Acc: 79.28,time : 111.13
Epoch-2/50 lr: 0.0001
5694/5694: [===============================>] - ETA 0.7sss
Train Loss: 0.1058, Train Acc: 75.15,Val Loss: 0.0851, Val Acc: 81.92,time : 112.78
Epoch-3/50 lr: 0.0001
5694/5694: [===============================>] - ETA 0.7sss
Train Loss: 0.0790, Train Acc: 82.17,Val Loss: 0.0466, Val Acc: 90.40,time : 113.28
Epoch-4/50 lr: 0.0001
5694/5694: [===============================>] - ETA 0.6sss
Train Loss: 0.0634, Train Acc: 86.49,Val Loss: 0.0504, Val Acc: 90.19,time : 113.59
Epoch-5/50 lr: 0.0001
5694/5694: [===============================>] - ETA 0.7sss
Train Loss: 0.0503, Train Acc: 89.62,Val Loss: 0.0638, Val Acc: 87.55,time : 114.61
Epoch-6/50 lr: 0.0001
5694/5694: [===============================>] - ETA 0.7sss
Train Loss: 0.0431, Train Acc: 91.22,Val Loss: 0.0592, Val Acc: 91.10,time : 113.94
Epoch-7/50 lr: 0.0001
5694/5694: